In [232]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [233]:
# Load the data from a CSV file
df = pd.read_csv("../data/all_v2_data.csv", sep=",")
df_pred= pd.read_csv("../data/september_to_predict.csv", sep=",")

In [234]:
df.dropna(inplace=True)

In [236]:
df

,Mnd,Ap,Hoyre,Frp,SV,SP,KrF,Venstre,MDG,Rodt,...,Rodt_reg_lag3,Rodt_reg_lag6,Andre_reg_lag3,Andre_reg_lag6,ledighet,ledig_trend3,ledig_trend6,styringsrente,styringsrente_trend3,styringsrente_trend6
5,6/30/2008,26.0,17.6,28.9,6.9,5.3,6.1,5.9,0.0,1.8,...,0.0,0.0,0.0,0.0,2.9,0.067,0.050000,5.54,0.076667,0.048333
6,7/31/2008,26.9,17.2,30.2,6.6,4.7,6.1,5.4,0.0,1.9,...,0.0,0.0,0.0,0.0,2.9,0.033,0.050000,5.75,0.083333,0.083333
7,8/31/2008,26.6,16.2,30.9,6.7,5.2,5.5,6.0,0.0,1.7,...,0.0,0.0,0.0,0.0,3.0,0.033,0.066667,5.75,0.070000,0.083333
8,9/30/2008,28.9,15.2,29.9,6.3,5.0,5.7,5.9,0.0,1.5,...,0.0,0.0,0.0,0.0,3.1,0.067,0.066667,5.75,0.000000,0.073333
9,10/31/2008,30.6,16.1,26.7,7.3,5.3,5.6,5.5,0.0,1.2,...,0.0,0.0,0.0,0.0,3.1,0.033,0.050000,5.45,-0.100000,-0.008333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,12/31/2024,17.2,21.2,25.4,9.4,5.3,3.9,5.3,3.1,5.8,...,0.0,0.0,0.0,0.0,4.1,0.033,0.016667,4.50,0.000000,0.000000
204,1/31/2025,19.1,22.2,24.1,8.0,5.9,3.4,4.5,3.3,5.6,...,0.0,0.0,0.0,0.0,4.1,0.000,0.016667,4.50,0.000000,0.000000
205,2/28/2025,24.8,18.4,24.6,7.4,6.6,3.0,4.2,2.4,5.4,...,0.0,0.0,0.0,0.0,4.1,0.000,0.016667,4.50,0.000000,0.000000
206,3/31/2025,28.1,18.8,22.7,7.2,4.9,3.1,4.3,2.7,4.6,...,0.0,0.0,0.0,0.0,4.0,-0.033,0.000000,4.50,0.000000,0.000000


In [237]:
df_pred

,Mnd,Ap,Hoyre,Frp,SV,SP,KrF,Venstre,MDG,Rodt,...,Rodt_reg_lag3,Rodt_reg_lag6,Andre_reg_lag3,Andre_reg_lag6,ledighet,ledig_trend3,ledig_trend6,styringsrente,styringsrente_trend3,styringsrente_trend6
0,9/30/2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,4,0,0,4,0,-0.033333


In [243]:
parties=['Ap', 'Hoyre', 'Frp', 'SV', 'SP', 'KrF', 'Venstre', 'MDG','Rodt', 'Andre']
predictions=[]

In [249]:
for party in parties:
    y=df[[party]]
    X=df[[f'{party}_var', 
          f'{party}_skandale',
          f'{party}_skandale_lag3', 
          f'{party}_skandale_lag6', 
          f'{party}_reg', f'{party}_reg_lag3',f'{party}_reg_lag6', 
       'ledighet', 'ledig_trend3', 'ledig_trend6', 'styringsrente',
       'styringsrente_trend3', 'styringsrente_trend6']]
    x_pred=df_pred[[ f'{party}_var', 
                    f'{party}_skandale',
                    f'{party}_skandale_lag3', 
                    f'{party}_skandale_lag6', 
          f'{party}_reg', f'{party}_reg_lag3',f'{party}_reg_lag6', 
       'ledighet', 'ledig_trend3', 'ledig_trend6', 'styringsrente',
       'styringsrente_trend3', 'styringsrente_trend6']]
    
    x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
    )
    pipeline = Pipeline([
    ("feature_selection", SelectFromModel(RandomForestRegressor(n_estimators=100, random_state=42), threshold="median")),
    ("regressor", RandomForestRegressor(n_estimators=1000, random_state=42))
    ])
    pipeline.fit(x_train, y_train.values.ravel())
    y_pred = pipeline.predict(x_test)
    r2 = r2_score(y_test, y_pred)
    
    
    p=pipeline.predict(x_pred)
    print(party + ": " + str(p[0]))
    predictions.append({'party': party, 'prediction': p[0], "r2_score": r2})

Ap: 30.43040000000024
Hoyre: 17.416199999999908
Frp: 19.343199999999833
SV: 6.576900000000065
SP: 7.481300000000016
KrF: 3.3995000000000317
Venstre: 3.977800000000024
MDG: 3.428700000000015
Rodt: 4.871999999999995
Andre: 4.796199999999947


In [250]:
pred_df = pd.DataFrame(predictions)
total = pred_df['prediction'].sum()
pred_df['prediction'] = (pred_df['prediction'] / total) * 100
pred_df['prediction'] = pred_df['prediction'].round(2)

In [248]:
pred_df

,party,prediction,r2_score
0,Ap,25.60,0.746131
1,Hoyre,23.10,0.684937
2,Frp,19.88,0.719545
3,SV,7.38,0.888477
4,SP,5.49,0.969987
5,KrF,3.40,0.867112
6,Venstre,4.28,0.757261
7,MDG,3.12,0.732925
8,Rodt,4.58,0.943925
9,Andre,3.17,0.917345


In [251]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import GradientBoostingRegressor

In [225]:
predictions=[]

In [252]:
for party in parties:
    y=df[[party]]
    X=df[[f'{party}_var', f'{party}_skandale',f'{party}_skandale_lag3', f'{party}_skandale_lag6', 
          f'{party}_reg', f'{party}_reg_lag3',f'{party}_reg_lag6', 
       'ledighet', 'ledig_trend3', 'ledig_trend6', 'styringsrente',
       'styringsrente_trend3', 'styringsrente_trend6']]
    x_pred=df_pred[[f'{party}_var', f'{party}_skandale',f'{party}_skandale_lag3', f'{party}_skandale_lag6', 
          f'{party}_reg', f'{party}_reg_lag3',f'{party}_reg_lag6', 
       'ledighet', 'ledig_trend3', 'ledig_trend6', 'styringsrente',
       'styringsrente_trend3', 'styringsrente_trend6']]
    
    x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
    )
    estimators = [
    ('rf', RandomForestRegressor(n_estimators=1000)),
    ('xgb', GradientBoostingRegressor(n_estimators=1000)),
    ('ridge', RidgeCV())
    ]
    stacking_model = StackingRegressor(estimators=estimators, final_estimator=RidgeCV())

    pipeline = Pipeline([
        ("feature_selection", SelectFromModel(RandomForestRegressor(n_estimators=100, random_state=42), threshold="median")),
        ("regressor", stacking_model)
    ])
    pipeline.fit(x_train, y_train.values.ravel())
    y_pred = pipeline.predict(x_test)
    r2 = r2_score(y_test, y_pred)
    
    
    p=pipeline.predict(x_pred)
    print(party + ": " + str(p[0]))
    predictions.append({'party': party, 'prediction': p[0], "r2_score": r2})

Ap: 30.113104871701186
Hoyre: 16.427822037656803
Frp: 18.99549418778765
SV: 6.7900199398367524
SP: 8.020115637922544
KrF: 3.469503339589547
Venstre: 4.0911777606318225
MDG: 3.555040969866084
Rodt: 4.701894032801723
Andre: 4.478308139024268


In [253]:
pred_df = pd.DataFrame(predictions)
total = pred_df['prediction'].sum()
pred_df['prediction'] = (pred_df['prediction'] / total) * 100
pred_df['prediction'] = pred_df['prediction'].round(2)

In [254]:
pred_df

,party,prediction,r2_score
0,Ap,8.74,0.746131
1,Hoyre,7.89,0.684937
2,Frp,6.79,0.719545
3,SV,2.52,0.888477
4,SP,1.88,0.969987
5,KrF,1.16,0.867112
6,Venstre,1.46,0.757261
7,MDG,1.06,0.732925
8,Rodt,1.56,0.943925
9,Andre,1.08,0.917345
